### Di seguito tutte le funzioni originali del modulo AlphaShapes, per poter comparare i loro tempi di esecuzione con le nuove versioni delle stesse funzioni

In [1]:
using LinearAlgebraicRepresentation
using Delaunay, Triangle
using Combinatorics, DataStructures
using Distributed, SharedArrays
Lar = LinearAlgebraicRepresentation
include("../src/AlphaShapes.jl") # nuovo modulo AlphaShapes

Main.AlphaShapes

In [2]:
function findCenter(P::Lar.Points)::Array{Float64,1}
    dim, n = size(P)
    @assert n > 0		"findCenter: at least one points is needed."
    @assert dim >= n-1	"findCenter: Too much points"

    @assert dim < 4		"findCenter: Function not yet Programmed."

    if n == 1
        center = P[:, 1]

    elseif n == 2
        #for each dimension
        center = (P[:, 1] + P[:, 2]) / 2

    elseif n == 3
        #https://www.ics.uci.edu/~eppstein/junkyard/circumcenter.html
        if dim == 2
            denom = 2 * Lar.det([ P[:, 2] - P[:, 1]  P[:, 3] - P[:, 1] ])
            deter = (P[:, 2] - P[:, 1]) * Lar.norm(P[:, 3] - P[:, 1])^2 -
                    (P[:, 3] - P[:, 1]) * Lar.norm(P[:, 2] - P[:, 1])^2
            numer = [- deter[2], deter[1]]
            center = P[:, 1] + numer / denom

        elseif dim == 3
            #circumcenter of a triangle in R^3
            numer = Lar.norm(P[:, 3] - P[:, 1])^2 * Lar.cross(
                        Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]),
                        P[:, 2] - P[:, 1]
                    ) +
                    Lar.norm(P[:, 2] - P[:, 1])^2 * Lar.cross(
                        P[:, 3] - P[:, 1],
                        Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]
                    )
            )
            denom = 2 * Lar.norm(
                Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1])
            )^2
            center = P[:, 1] + numer / denom
        end

    elseif n == 4 #&& dim = 3
        # https://people.sc.fsu.edu/~jburkardt/presentations
        #	/cg_lab_tetrahedrons.pdf
        # page 6 (matrix are transposed)
        α = Lar.det([P; ones(1, 4)])
        sq = sum(abs2, P, dims = 1)
        Dx = Lar.det([sq; P[2:2,:]; P[3:3,:]; ones(1, 4)])
        Dy = Lar.det([P[1:1,:]; sq; P[3:3,:]; ones(1, 4)])
        Dz = Lar.det([P[1:1,:]; P[2:2,:]; sq; ones(1, 4)])
        center = [Dx; Dy; Dz]/2α
    end

    return center
end

findCenter (generic function with 1 method)

In [3]:
function findClosestPoint(
        Psimplex::Lar.Points, P::Lar.Points;
        metric = "circumcenter"
    )::Union{Int64, Nothing}

    @assert metric ∈ ["circumcenter", "dd"] "findClosestPoint: available metrics are
        `circumcenter` and `dd`."

    simplexDim = size(Psimplex, 2)
    @assert simplexDim <= size(Psimplex, 1) "findClosestPoint: Cannot add
    another point to the simplex."

    @assert (m = size(P, 2)) != 0 "findClosestPoint: No Points in `P`."

    radlist = SharedArray{Float64}(m)
    for col = 1 : m
        r, c = findRadius([Psimplex P[:,col]], true)
        sameSign = (
            r == Inf ||
            metric != "dd" ||
            isempty(oppositeHalfSpacePoints(
                [Psimplex P[:,col]], Psimplex, c
            ))
        )
        radlist[col] = ((-1)^(1 + sameSign)) * r
    end

    radius, closestidx = findmin(radlist)

    if radius == Inf
        closestidx = nothing
    end

    return closestidx

end


findClosestPoint (generic function with 1 method)

In [4]:
function findMedian(P::Lar.Points, ax::Int64)::Float64
    xp = sort(unique(P[ax, :]))
    if length(xp) == 1
        median = xp[1]
    else
        idx = Int64(floor(length(xp)/2))
        median = (xp[idx] + xp[idx+1])/2
    end
    return median
end

findMedian (generic function with 1 method)

In [5]:
function findRadius(
        P::Lar.Points, center=false; digits=64
    )::Union{Float64, Tuple{Float64, Array{Float64,1}}}

    c = findCenter(P)
    if any(isnan, c)
        r = Inf
    else
        r = round(
            findmin([Lar.norm(c - P[:, i]) for i = 1 : size(P, 2)])[1],
            digits = digits
        )
    end
    if center
        return r, c
    end
    return r
end

findRadius (generic function with 2 methods)

In [6]:
function matrixPerturbation(
        M::Array{Float64,2};
        atol=1e-10, row = [0], col = [0]
    )::Array{Float64,2}

    if atol == 0.0
        println("Warning: no perturbation has been performed.")
        return M
    end

    if row == [0]
        row = [i for i = 1 : size(M, 1)]
    end
    if col == [0]
        col = [i for i = 1 : size(M, 2)]
    end

    N = copy(M)
    perturbation = mod.(rand(Float64, length(row), length(col)), 2*atol).-atol
    N[row, col] = M[row, col] + perturbation
    return N
end

matrixPerturbation (generic function with 1 method)

In [7]:
function oppositeHalfSpacePoints(
        P::Lar.Points,
        face::Array{Float64,2},
        point::Array{Float64,1}
    )::Array{Int64,1}

    dim, n = size(P)
    noV = size(face, 2)
    @assert dim <= 3 "oppositeHalfSpacePoints: Not yet coded."
    @assert noV == dim "oppositeHalfSpacePoints:
        Cannot determine opposite to non hyperplanes."
    if dim == 1
        threshold = face[1]
        if point[1] < threshold
            opposite = [i for i = 1 : n if P[1, i] > threshold]
        else
            opposite = [i for i = 1 : n if P[1, i] < threshold]
        end
    elseif dim == 2
        if (Δx = face[1, 1] - face[1, 2]) != 0.0
            m = (face[2, 1] - face[2, 2]) / Δx
            q = face[2, 1] - m * face[1, 1]
            # false = under the line, true = over the line
            @assert point[2] ≠ m * point[1] + q "oppositeHalfSpacePoints,
                the point belongs to the face"
            side = sign(m * point[1] + q - point[2])
            opposite =
                [i for i = 1 : n if side * (m * P[1, i] + q - P[2, i]) < 0]
        else
            q = face[1, 1]
            side = sign(point[1] - q)
            opposite = [i for i = 1 : n if side * (P[1, i] - q) < 0]
        end


    elseif dim == 3
        axis = Lar.cross(
            face[:, 2] - face[:, 1],
            face[:, 3] - face[:, 1]
        )
        off = Lar.dot(axis, face[:, 1])
        position = Lar.dot(point, axis)
        if position < off
            opposite = [i for i = 1:size(P, 2) if Lar.dot(P[:,i], axis) > off]
        else
            opposite = [i for i = 1:size(P, 2) if Lar.dot(P[:,i], axis) < off]
        end
    end

    return [
        i for i in opposite
        if sum([P[:, i] == face[:, j] for j = 1 : noV]) == 0
    ]
end

oppositeHalfSpacePoints (generic function with 1 method)

In [8]:
function planarIntersection(
        P::Lar.Points,
        face::Array{Int64,1},
        axis::Int64,
        off::Float64
    )::Int64

    pos = [P[axis, i] > off for i in face]

    if sum([P[axis, i] == off for i in face]) == length(pos)
        position = 0 # face coplanar with axis
    elseif sum(pos) == 0
        position = -1
    elseif sum(pos) == length(pos)
        position = +1
    else
        position = 0
    end

    return position
end

planarIntersection (generic function with 1 method)

In [9]:
function simplexFaces(σ::Array{Int64,1})::Array{Array{Int64,1},1}
    sort!(sort!.(collect(Combinatorics.combinations(σ, length(σ)-1))))
end

simplexFaces (generic function with 1 method)

In [10]:
function vertexInCircumball(
        P::Lar.Points,
        α_char::Float64,
        point::Array{Float64,2}
    )::Bool

    center = findCenter(P)
    return Lar.norm(point - center) <= α_char
end

vertexInCircumball (generic function with 1 method)

In [11]:
function delaunayWall(
        P::Lar.Points,
        ax = 1,
        Pblack = Float64[],
        AFL = Array{Int64,1}[],
        tetraDict = DataStructures.Dict{Array{Int64,1},Array{Float64,1}}();
        DEBUG = false
    )::Lar.Cells

    if DEBUG @show "Delaunay Wall with parameters" P ax AFL tetraDict end

    # 0 - Data Reading and Container definition
    DT = Array{Int64,1}[]		# Delaunay Triangulation
    AFLα = Array{Int64,1}[]		# (d-1)faces intersecting the Wall
    AFLplus = Array{Int64,1}[]  # (d-1)faces in positive Wall half-space
    AFLminus = Array{Int64,1}[] # (d-1)faces in positive Wall half-space
    off = findMedian(P, ax)
    if !isempty(Pblack) Pext = [P Pblack] else Pext = copy(P) end

    # 1 - Determine first simplex (if necessary)
    if isempty(AFL)
        @assert isempty(Pblack) "delaunayWall: If AFL is empty => Pblack must be"
        @assert isempty(tetraDict) "delaunayWall: If AFL is empty => tetraDict must be"
        σ = sort(firstDeWallSimplex(P, ax, off, DEBUG = DEBUG))
        push!(DT, σ)
        AFL = simplexFaces(σ)
        updateTetraDict!(P, tetraDict, AFL, σ)
    else
        @assert !isempty(Pblack) "delaunayWall: Data missing - Pblack"
        @assert !isempty(AFL) "delaunayWall: Data missing - AFL"
        @assert !isempty(tetraDict) "delaunayWall: Data missing - tetraDict"
    end

    # 2 - Build `AFL*` according to the axis `ax` with constant term `off`
    updateAFL!(
        P, AFL, AFLα, AFLplus, AFLminus, ax, off, DEBUG = DEBUG
    )

    # 4 - Build simplex Wall
    while !isempty(AFLα)
        # if face ∈ keys(tetraDict) oppoint = tetraDict[face]
        # else Pselection = setdiff([i for i = 1 : n], face) end
        σ = findWallSimplex(
                Pext, AFLα[1], tetraDict[AFLα[1]], size(P, 2), DEBUG = DEBUG
            )
        if σ != nothing && σ ∉ DT
            push!(DT, σ)
            AFL = simplexFaces(σ)
            updateTetraDict!(P, tetraDict, AFL, σ)
            # Split σ's Faces according in semi-spaces
            updateAFL!(
                P, AFL, AFLα, AFLplus, AFLminus, ax, off, DEBUG=DEBUG
            )
        else
            @assert updatelist!(AFLα, AFLα[1]) == false "delaunayWall:
                Something unespected happends while removing a face."
        end
    end

    # 5 - Change the axis `ax` and repeat until there are no faces but exposed.
    #      A.K.A. Divide & Conquer phase.
    if !isempty(AFLminus)
        union!(DT, recursiveDelaunayWall(
            P, Pblack, tetraDict, AFLminus, ax, off, false; DEBUG = DEBUG
        ))
    end
    if !isempty(AFLplus)
        union!(DT, recursiveDelaunayWall(
            P, Pblack, tetraDict, AFLplus, ax, off, true; DEBUG = DEBUG
        ))
    end

    return DT
end

delaunayWall (generic function with 5 methods)

In [12]:
function findWallSimplex(
        P::Lar.Points,
        face::Array{Int64,1},
        oppoint::Array{Float64,1},
        blackidx = size(P, 2);
        DEBUG = false
    )::Union{Array{Int64,1}, Nothing}

    if DEBUG @show "find Wall Simplex of" face oppoint end
    # Find the points in the halfspace defined by `face` that do not
    #  containsother the other point of the simplex.
    Pselection =
        oppositeHalfSpacePoints(P, P[:, face], oppoint)

    if DEBUG @show Pselection end

    # If there are no such points than the face is part of the convex hull.
    if isempty(Pselection)
        return nothing
    end

    # Find the Closest Point in the other halfspace with respect to σ
    #  according to dd-distance.
    idxbase = Pselection[ findClosestPoint(
        P[:, face], P[:, Pselection], metric = "dd"
    ) ]

    # @assert !isnothing(idxbase)
    # if isnothing(idxbase)
    # 	return nothing
    # end

    # It prevent from adding the same simplex again (cause it has been
    #  determined in a previous recursive call in the stacktrace).
    if idxbase > blackidx
        if DEBUG println("Excluding $face cause simplex already inside.") end
        return nothing
    end

    σ = sort([face; idxbase])
    if DEBUG @show "Found face" σ end

    # Check the simplex correctness
    radius, center = findRadius(P[:, σ], true)
    for i = 1 : size(P, 2)
        if Lar.norm(center - P[:, i]) < radius - 1.e-14
            # @assert i ∉ Pselection "ERROR: Numerical error
            # 	evaluating minimum radius for $σ"
            if DEBUG println("$σ discarded due to a closer point.") end
            return nothing
        end
    end

    return σ
end



findWallSimplex (generic function with 2 methods)

In [13]:
function firstDeWallSimplex(
        P::Lar.Points,
        ax::Int64,
        off::Float64;
        DEBUG = false
    )::Array{Int64,1}

    dim = size(P, 1)
    n = size(P, 2)

    if DEBUG println("Determine first Simplex with ax = $ax") end
    # the first point of the simplex is the one with coordinate `ax` maximal
    #  such that it is less than `off` (closer to α from minus)

    Pselection = findall(x -> x < off, P[ax, :])

    # it gives an error if no point are less than `off`
    #  in fact it means that all the points are located on the median,
    #  with respect to `ax`.
    @assert !isempty(Pselection) "firstDeWallSimplex: not able to build the first Delaunay
        dimplex; all the points have the same `ax` coordinate."
    newidx = Pselection[findmax(P[ax, Pselection])[2]]
    # indices will store the indices of the simplex ...
    indices = [newidx]                      #Array{Int64,1}
    # ... and `Psimplex` will store the corresponding points
    Psimplex = P[:, newidx][:,:]    #Array{Float64,2}

    # the second point must be seeken across those with coordinate `ax`
    #  grater than `off`
    Pselection = findall(x -> x > off, P[ax, :])

    for d = 1 : dim
        idxbase = findClosestPoint(Psimplex, P[:, Pselection])
        @assert !isnothing(idxbase) "firstDeWallSimplex:
            not able to determine first Delaunay Simplex"
        newidx = Pselection[idxbase]
        indices = [indices; newidx]
        Psimplex = [Psimplex P[:, newidx]]
        Pselection = [i for i = 1 : n if i ∉ indices]
    end

    # Correctness check
    radius, center = findRadius(Psimplex, true)
    for i = 1 : n
        @assert Lar.norm(center - P[:, i]) >= radius "firstDeWallSimplex:
            Unable to find first Simplex."
    end

    if DEBUG println("First Simplex = $indices") end

    return indices
end

firstDeWallSimplex (generic function with 1 method)

In [14]:
function recursiveDelaunayWall(
        P::Lar.Points,
        Pblack::Array{Float64},
        tetraDict::DataStructures.Dict{Array{Int64,1},Array{Float64,1}},
        AFL::Array{Array{Int64,1},1},
        ax::Int64,
        off::Float64,
        positive::Bool;
        DEBUG = false
    )::Lar.Cells

    #DEBUG = true

    dim, n = size(P)
    newaxis = mod(ax, dim) + 1

    if DEBUG println("Divide Plus/Minus $positive") end

    Psubset = findall(x -> (x > off) == positive, P[ax, :])
    blacklist = setdiff(unique([(keys(tetraDict)...)...]), Psubset)
    if !isempty(Pblack)
        Pblack = [Pblack P[:, blacklist]]
    else
        Pblack = P[:, blacklist]
    end

    if DEBUG println("Step In") end

    DT = delaunayWall(
            P[:, Psubset],
            newaxis,
            Pblack,
            [[findall(Psubset.==p)[1] for p in σ] for σ in AFL],
            Dict([
                [findall(Psubset.==p)[1] for p in k] => v
                for (k,v) in tetraDict
                    if k ⊆ Psubset
            ]),
            DEBUG = DEBUG
        )

    if DEBUG @show "Step Out with " DT end

    return [[Psubset[i] for i in σ] for σ in DT]
end


recursiveDelaunayWall (generic function with 1 method)

In [15]:
function updateAFL!(
        P::Lar.Points,
        newσ::Array{Array{Int64,1},1},
        AFLα::Array{Array{Int64,1},1},
        AFLplus::Array{Array{Int64,1},1},
        AFLminus::Array{Array{Int64,1},1},
        ax::Int64, off::Float64;
        DEBUG = false
    )::Bool

    for face in newσ
        inters = planarIntersection(P, face, ax, off)
        if inters == 0 # intersected by plane α
            updatelist!(AFLα, face)
        elseif inters == -1 # in NegHalfspace(α)
            updatelist!(AFLminus, face)
        elseif inters == 1 # in PosHalfspace(α)
            updatelist!(AFLplus, face)
        else
            return false
        end
    end

    if DEBUG @show AFLα AFLminus AFLplus end

    return true

end


updateAFL! (generic function with 1 method)

In [16]:
function updatelist!(list, element)::Bool
    if element ∈ list
        setdiff!(list, [element])
        return false
    else
        push!(list, element)
        return true
    end
end

updatelist! (generic function with 1 method)

In [17]:
function updateTetraDict!(
        P::Lar.Points,
        tetraDict::DataStructures.Dict{Array{Int64,1},Array{Float64,1}},
        AFL::Array{Array{Int64,1},1},
        σ::Array{Int64,1}
    )::Nothing
    for cell in AFL
        point = setdiff(σ, cell)
        @assert length(point) == 1 "updateTetraDict!: Error during update of TetraDict $σ, $cell"
        tetraDict[ cell ] = P[:, point[1]]
    end
end


updateTetraDict! (generic function with 1 method)

In [18]:
function alphaFilter(
        V::Lar.Points,
        DT = Array{Int64,1}[];
        digits=64
    )::DataStructures.SortedDict{}

    dim = size(V, 1)
    filtration = DataStructures.SortedDict{Array{Int64,1},Float64}()

    # 1 - Each point => alpha_char = 0.
    for i = 1 : size(V, 2)
        insert!(filtration, [i], 0.)
    end

    # 2 - Delaunay triangulation of ``V``
    if isempty(DT)
        DT = delaunayTriangulation(V)
    end

    n_upsimplex = length(DT)

    # 3 - process all upper simplex
    ind = 1
    for upper_simplex in DT
        if ind % 500000 == 0
            println(ind," simplices processed of ", n_upsimplex)
        end
        processuppersimplex(V,upper_simplex,filtration; digits = digits)
        ind = ind + 1
    end

    return filtration
end

alphaFilter (generic function with 2 methods)

In [19]:
function processuppersimplex(
        V::Lar.Points,
        up_simplex::Array{Int64,1},
        filtration::DataStructures.SortedDict{};
        digits=64
        )

    α_char = findRadius(V[:, up_simplex], digits=digits);
    insert!(filtration, up_simplex, α_char)

    d = length(up_simplex)-1
    if d > 1
        # It gives back combinations in natural order
        newsimplex = collect(Combinatorics.combinations(up_simplex,d))
        for lowsimplex in newsimplex
            processlowsimplex(V, up_simplex, lowsimplex, filtration; digits=digits)
        end
    end
end

processuppersimplex (generic function with 1 method)

In [20]:
function processlowsimplex(
    V::Lar.Points,
    up_simplex::Array{Int64,1},
    lowsimplex::Array{Int64,1},
    filtration::DataStructures.SortedDict{};
    digits=64)

    α_char = findRadius(V[:, lowsimplex], digits=digits)
    point = V[:, setdiff(up_simplex, lowsimplex)]

    if vertexInCircumball(V[:, lowsimplex], α_char, point)
        filtration[lowsimplex] = filtration[up_simplex]

    elseif !haskey(filtration, lowsimplex)
        filtration[lowsimplex] = α_char

    end

    d = length(lowsimplex)-1
    if d > 1
        # It gives back combinations in natural order
        newsimplex = collect(Combinatorics.combinations(lowsimplex,d))
        for simplex in newsimplex
             processlowsimplex(V, lowsimplex, simplex, filtration, digits=digits)
        end
    end
end


processlowsimplex (generic function with 1 method)

In [21]:
function alphaSimplex(
        V::Lar.Points,
        filtration::DataStructures.SortedDict{},
        α_threshold::Float64
    )::Array{Lar.Cells,1}

    dim = size(V, 1)
    # [VV, EV, FV, ...]
    simplexCollection = [ Array{Array{Int64,1},1}() for i = 1 : dim+1 ]

    for (k, v) in filtration
        if v <= α_threshold
            push!(simplexCollection[length(k)], k)
        end
    end

    sort!.(simplexCollection)

    return simplexCollection
end

alphaSimplex (generic function with 1 method)

In [22]:
function delaunayTriangulation(V::Lar.Points)::Lar.Cells
    dim = size(V, 1)
    @assert dim > 0 "delaunayTriangulation: V do not contains points."
    @assert dim < 4 "delaunayTriangulation: Function not yet Programmed."

    if dim == 1
        vertices = vcat(V...)
        p = sortperm(vertices)
        upper_simplex = [[p[i],p[i+1]] for i=1:length(p)-1]

    elseif dim == 2
        vertices = convert(Array{Float64,2},V')
        points_map = Array{Int64,1}(collect(1:1:size(vertices)[1]))
        @assert size(vertices, 1) > 3
        upper_simplex = Triangle.basic_triangulation(vertices, points_map)

    elseif dim == 3
        upper_simplex = delaunayWall(V)
    end

    sort!.(upper_simplex)

    return sort(upper_simplex)
end

delaunayTriangulation (generic function with 1 method)

# Esempio 3D (utilizzando le funzioni del vecchio modulo)

In [23]:
using LinearAlgebraicRepresentation, ViewerGL
using BenchmarkTools
using Distributed
GL = ViewerGL
filename = "../examples/examples3D/OBJ/teapot.obj";

W, EVs, FVs = Lar.obj2lar(filename);
WW = [[i] for i = 1:size(W, 2)];
V, VV = Lar.apply(Lar.r(pi / 2, 0, 0), (W, WW)); #object rotated

points = convert(Lar.Points, V')
#=
GL.VIEW([
    GL.GLPoints(points)
    GL.GLAxis(GL.Point3d(-1, -1, -1), GL.Point3d(1, 1, 1))
]);
=#

529×3 Array{Float64,2}:
 20.2617   3.9956  22.3469
 19.2515  -1.1209  22.3469
 19.0837  -1.0495  23.0343
 20.0799   3.9956  23.0343
 19.2783  -1.1323  23.2634
 20.2908   3.9956  23.2634
 19.6742  -1.3008  23.0343
 20.72     3.9956  23.0343
 20.1104  -1.4864  22.3469
 21.1929   3.9956  22.3469
 16.4814  -5.2597  22.3469
 16.3523  -5.1306  23.0343
 16.5021  -5.2804  23.2634
  ⋮                
  8.8935   7.9146  24.3109
 10.2411  11.082   23.7435
 11.4517  13.9273  23.1761
 11.9771  15.162   22.3469
 10.2423   7.0118  24.3109
 12.6801   9.4496  23.7435
 14.87    11.6395  23.1761
 15.8203  12.5898  22.3469
 11.1451   5.663   24.3109
 14.3125   7.0107  23.7435
 17.1578   8.2213  23.1761
 18.3925   8.7466  22.3469

In [24]:
@btime filtration = alphaFilter(V); # 8.448 s (51918223 allocations: 4.07 GiB)

  7.167 s (51972813 allocations: 4.07 GiB)


In [25]:
@benchmark filtration = alphaFilter(V);

In [26]:
@code_warntype alphaFilter(V);

Variables
  #self#::Core.Compiler.Const(alphaFilter, false)
  V::Array{Float64,2}

Body::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
1 ─ %1 = Core.apply_type(Main.Array, Main.Int64, 1)::Core.Compiler.Const(Array{Int64,1}, false)
│   %2 = Base.getindex(%1)::Array{Array{Int64,1},1}
│   %3 = (#self#)(V, %2)::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
└──      return %3


In [27]:
filtration = alphaFilter(V);

In [28]:
@btime VV, EV, FV, TV = alphaSimplex(V, filtration, 3.7) # 2.156 ms (55 allocations: 205.47 KiB)

  2.008 ms (55 allocations: 205.47 KiB)


4-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [520], [521], [522], [523], [524], [525], [526], [527], [528], [529]]
 [[1, 2], [1, 3], [1, 4], [1, 8], [1, 10], [1, 76], [1, 77], [1, 82], [1, 472], [1, 473]  …  [523, 524], [523, 527], [523, 528], [524, 525], [524, 528], [524, 529], [525, 529], [526, 527], [527, 528], [528, 529]]
 [[1, 2, 3], [1, 2, 8], [1, 2, 10], [1, 2, 472], [1, 3, 4], [1, 3, 8], [1, 3, 472], [1, 4, 8], [1, 4, 77], [1, 4, 472]  …  [519, 522, 523], [520, 521, 524], [520, 523, 524], [521, 524, 525], [522, 523, 527], [522, 526, 527], [523, 524, 528], [523, 527, 528], [524, 525, 529], [524, 528, 529]]
 [[1, 2, 3, 8], [1, 2, 3, 472], [1, 2, 8, 10], [1, 3, 4, 8], [1, 3, 4, 472], [1, 4, 8, 77], [1, 4, 77, 529], [1, 4, 472, 473], [1, 4, 473, 529], [1, 8, 10, 76]  …  [459, 460, 518, 522], [459, 460, 522, 526], [460, 461, 464, 465], [460, 461, 465, 526], [460, 461, 518, 522], [460, 461, 522, 526], [460, 463, 464, 526], [460, 

In [29]:
@benchmark VV, EV, FV, TV = alphaSimplex(V, filtration, 3.7)

BenchmarkTools.Trial: 
  memory estimate:  205.47 KiB
  allocs estimate:  55
  --------------
  minimum time:     2.080 ms (0.00% GC)
  median time:      2.400 ms (0.00% GC)
  mean time:        2.647 ms (1.09% GC)
  maximum time:     13.189 ms (0.00% GC)
  --------------
  samples:          1886
  evals/sample:     1

In [30]:
@code_warntype alphaSimplex(V, filtration, 3.7)

Variables
  #self#::Core.Compiler.Const(alphaSimplex, false)
  V::Array{Float64,2}
  filtration::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
  α_threshold::Float64
  #56::var"#56#57"
  dim::Int64
  simplexCollection::Array{Array{Array{Int64,1},1},1}
  @_8::Union{Nothing, Tuple{Pair{Array{Int64,1},Float64},DataStructures.SAIterationState}}
  k::Array{Int64,1}
  v::Float64
  @_11::Int64

Body::Array{_A,1} where _A
1 ─ %1  = Base.getproperty(Main.Lar, :Cells)::Any
│   %2  = Core.apply_type(Main.Array, %1, 1)::Type{Array{_A,1}} where _A
│         (dim = Main.size(V, 1))
│         (#56 = %new(Main.:(var"#56#57")))
│   %5  = #56::Core.Compiler.Const(var"#56#57"(), false)
│   %6  = (dim + 1)::Int64
│   %7  = (1:%6)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│   %8  = Base.Generator(%5, %7)::Core.Compiler.PartialStruct(Base.Generator{UnitRange{Int64},var"#56#57"}, Any[Core.Compiler.Const(var"#56#57"(), false), Core.Compiler.Parti

In [31]:
VV, EV, FV, TV = alphaSimplex(V, filtration, 3.7)
#=
GL.VIEW([
    GL.GLGrid(V, EV, GL.COLORS[1], 0.6) # White
    GL.GLGrid(V, FV, GL.COLORS[2], 0.3) # Red
    GL.GLGrid(V, TV, GL.COLORS[3], 0.3) # Green
]);
=#
filter_key = sort(unique(values(filtration)))

granular = 10

reduced_filter =
    filter_key[sort(abs.(rand(Int, granular) .% length(filter_key)))]
reduced_filter = [reduced_filter; max(filter_key...)]

α=0.0
for α in reduced_filter
    @show α
    @btime VVV, EEV, FFV, TTV = alphaSimplex(V, filtration, α) # 772.066 μs (20 allocations: 19.34 KiB) in media
end

α = 1.968405421061437
  792.433 μs (20 allocations: 19.34 KiB)
α = 2.017671772117555
  796.021 μs (20 allocations: 19.34 KiB)
α = 2.4890822142194553
  783.538 μs (20 allocations: 19.34 KiB)
α = 3.007899400831171
  784.376 μs (20 allocations: 19.34 KiB)
α = 4.89501222041184
  783.134 μs (20 allocations: 19.34 KiB)
α = 4.896038200462319
  788.548 μs (20 allocations: 19.34 KiB)
α = 9.19116012889933
  781.332 μs (20 allocations: 19.34 KiB)
α = 10.408317716047518
  791.985 μs (20 allocations: 19.34 KiB)
α = 11.1311563063905
  821.066 μs (20 allocations: 19.34 KiB)
α = 11.155073381217914
  792.667 μs (20 allocations: 19.34 KiB)
α = Inf
  786.494 μs (20 allocations: 19.34 KiB)


# Esempio 3D (utilizzando le funzioni del nuovo modulo)

In [32]:
using LinearAlgebraicRepresentation, ViewerGL
using BenchmarkTools
using Distributed
GL = ViewerGL
filename = "../examples/examples3D/OBJ/teapot.obj";

W, EVs, FVs = Lar.obj2lar(filename);
WW = [[i] for i = 1:size(W, 2)];
V, VV = Lar.apply(Lar.r(pi / 2, 0, 0), (W, WW)); #object rotated

points = convert(Lar.Points, V')
#=
GL.VIEW([
    GL.GLPoints(points)
    GL.GLAxis(GL.Point3d(-1, -1, -1), GL.Point3d(1, 1, 1))
]);
=#

529×3 Array{Float64,2}:
 20.2617   3.9956  22.3469
 19.2515  -1.1209  22.3469
 19.0837  -1.0495  23.0343
 20.0799   3.9956  23.0343
 19.2783  -1.1323  23.2634
 20.2908   3.9956  23.2634
 19.6742  -1.3008  23.0343
 20.72     3.9956  23.0343
 20.1104  -1.4864  22.3469
 21.1929   3.9956  22.3469
 16.4814  -5.2597  22.3469
 16.3523  -5.1306  23.0343
 16.5021  -5.2804  23.2634
  ⋮                
  8.8935   7.9146  24.3109
 10.2411  11.082   23.7435
 11.4517  13.9273  23.1761
 11.9771  15.162   22.3469
 10.2423   7.0118  24.3109
 12.6801   9.4496  23.7435
 14.87    11.6395  23.1761
 15.8203  12.5898  22.3469
 11.1451   5.663   24.3109
 14.3125   7.0107  23.7435
 17.1578   8.2213  23.1761
 18.3925   8.7466  22.3469

In [33]:
@btime filtration = AlphaShapes.alphaFilter(V); # 5.039 s (35330571 allocations: 3.80 GiB)

  4.886 s (33369327 allocations: 3.73 GiB)


In [34]:
@benchmark filtration = AlphaShapes.alphaFilter(V);

In [35]:
@code_warntype AlphaShapes.alphaFilter(V);

Variables
  #self#::Core.Compiler.Const(Main.AlphaShapes.alphaFilter, false)
  V::Array{Float64,2}

Body::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
1 ─ %1 = Core.apply_type(Main.AlphaShapes.Array, Main.AlphaShapes.Int64, 1)::Core.Compiler.Const(Array{Int64,1}, false)
│   %2 = Base.getindex(%1)::Array{Array{Int64,1},1}
│   %3 = (#self#)(V, %2)::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
└──      return %3


In [36]:
filtration = AlphaShapes.alphaFilter(V);

In [37]:
@btime VV, EV, FV, TV = AlphaShapes.alphaSimplex(V, filtration, 3.7) # 2.110 ms (55 allocations: 205.47 KiB)

  1.930 ms (55 allocations: 205.47 KiB)


4-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [520], [521], [522], [523], [524], [525], [526], [527], [528], [529]]
 [[1, 2], [1, 3], [1, 4], [1, 8], [1, 10], [1, 76], [1, 77], [1, 82], [1, 472], [1, 473]  …  [523, 524], [523, 527], [523, 528], [524, 525], [524, 528], [524, 529], [525, 529], [526, 527], [527, 528], [528, 529]]
 [[1, 2, 3], [1, 2, 8], [1, 2, 10], [1, 2, 472], [1, 3, 4], [1, 3, 8], [1, 3, 472], [1, 4, 8], [1, 4, 77], [1, 4, 472]  …  [519, 522, 523], [520, 521, 524], [520, 523, 524], [521, 524, 525], [522, 523, 527], [522, 526, 527], [523, 524, 528], [523, 527, 528], [524, 525, 529], [524, 528, 529]]
 [[1, 2, 3, 8], [1, 2, 3, 472], [1, 2, 8, 10], [1, 3, 4, 8], [1, 3, 4, 472], [1, 4, 8, 77], [1, 4, 77, 529], [1, 4, 472, 473], [1, 4, 473, 529], [1, 8, 10, 76]  …  [459, 460, 518, 522], [459, 460, 522, 526], [460, 461, 464, 465], [460, 461, 465, 526], [460, 461, 518, 522], [460, 461, 522, 526], [460, 463, 464, 526], [460, 

In [38]:
@benchmark VV, EV, FV, TV = AlphaShapes.alphaSimplex(V, filtration, 3.7)

BenchmarkTools.Trial: 
  memory estimate:  205.47 KiB
  allocs estimate:  55
  --------------
  minimum time:     1.902 ms (0.00% GC)
  median time:      2.152 ms (0.00% GC)
  mean time:        2.257 ms (1.38% GC)
  maximum time:     10.956 ms (78.93% GC)
  --------------
  samples:          2214
  evals/sample:     1

In [39]:
@code_warntype AlphaShapes.alphaSimplex(V, filtration, 3.7)

Variables
  #self#::Core.Compiler.Const(Main.AlphaShapes.alphaSimplex, false)
  V::Array{Float64,2}
  filtration::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
  α_threshold::Float64
  #12::Main.AlphaShapes.var"#12#14"{Array{Float64,2},SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering},Float64}

Body::Array{Array{Array{Int64,1},1},1}
1 ─ %1  = LinearAlgebraicRepresentation.Cells::Core.Compiler.Const(Array{Array{Int64,1},1}, false)
│   %2  = Core.apply_type(Main.AlphaShapes.Array, %1, 1)::Core.Compiler.Const(Array{Array{Array{Int64,1},1},1}, false)
│   %3  = Main.AlphaShapes.:(var"#12#14")::Core.Compiler.Const(Main.AlphaShapes.var"#12#14", false)
│   %4  = Core.typeof(V)::Core.Compiler.Const(Array{Float64,2}, false)
│   %5  = Core.typeof(filtration)::Core.Compiler.Const(SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}, false)
│   %6  = Core.typeof(α_threshold)::Core.Compiler.Const(Float64, false)
│   %7  = Core.apply_type(%3, %4, %5, %6)::Core.Com

In [40]:
VV, EV, FV, TV = AlphaShapes.alphaSimplex(V, filtration, 3.7)
#=
GL.VIEW([
    GL.GLGrid(V, EV, GL.COLORS[1], 0.6) # White
    GL.GLGrid(V, FV, GL.COLORS[2], 0.3) # Red
    GL.GLGrid(V, TV, GL.COLORS[3], 0.3) # Green
]);
=#
filter_key = sort(unique(values(filtration)))

granular = 10

reduced_filter =
    filter_key[sort(abs.(rand(Int, granular) .% length(filter_key)))]
reduced_filter = [reduced_filter; max(filter_key...)]

α=0.0
for α in reduced_filter
    @show α
    @btime VVV, EEV, FFV, TTV = AlphaShapes.alphaSimplex(V, filtration, α) # 685.523 μs (20 allocations: 19.34 KiB) in media
end

α = 0.5258829418615056
  667.246 μs (20 allocations: 19.34 KiB)
α = 1.7346789561711407
  695.315 μs (20 allocations: 19.34 KiB)
α = 1.8293452804213848
  653.233 μs (20 allocations: 19.34 KiB)
α = 2.225168094478342
  630.138 μs (20 allocations: 19.34 KiB)
α = 2.249946519255121
  670.155 μs (20 allocations: 19.34 KiB)
α = 2.8022667377531554
  625.289 μs (20 allocations: 19.34 KiB)
α = 4.4652767434338605
  668.286 μs (20 allocations: 19.34 KiB)
α = 4.895421873351256
  686.908 μs (20 allocations: 19.34 KiB)
α = 5.0655380118735955
  647.961 μs (20 allocations: 19.34 KiB)
α = 6.394084225387963
  633.560 μs (20 allocations: 19.34 KiB)
α = Inf
  721.837 μs (20 allocations: 19.34 KiB)


# Esempio 2D (utilizzando le funzioni del vecchio modulo)

In [41]:
"""
    pointsRand(V, VV, n, m)

Generate random points inside and otuside `(V, VV)`.
"""
function pointsRand(
        V::Lar.Points, EV::Lar.Cells, n = 1000, m = 0
    )::Tuple{Lar.Points, Lar.Points, Lar.Cells, Lar.Cells}
    classify = Lar.pointInPolygonClassification(V, EV)
    Vi = [0;0]
    Ve = [0;0]
    k1 = 0
    k2 = 0
    while k1 < n || k2 < m
        queryPoint = [rand();rand()]
        inOut = classify(queryPoint)

        if k1 < n && inOut == "p_in"
            Vi = hcat(Vi, queryPoint)
            k1 = k1 + 1;
        end
        if k2 < m && inOut == "p_out"
            Ve = hcat(Ve, queryPoint)
            k2 = k2 + 1;
        end
    end
    VVi = [[i] for i = 1 : n]
    VVe = [[i] for i = 1 : m]
    return Vi[:,2:end], Ve[:,2:end], VVi, VVe
end

filename = "../examples/examples2D/svg_files/Lar2.svg";

V,EV = Lar.svg2lar(filename);

Vi, Ve, VVi, VVe = pointsRand(V, EV, 1000, 10000);

#=
GL.VIEW([
 	GL.GLGrid(Vi, VVi, GL.COLORS[1], 1)
 	GL.GLGrid(Ve, VVe, GL.COLORS[12], 1)
])
=#

In [42]:
@btime filtration = alphaFilter(Vi); # 38.343 ms (371714 allocations: 25.58 MiB)

  35.687 ms (377142 allocations: 25.64 MiB)


In [43]:
@benchmark filtration = alphaFilter(Vi);

In [44]:
@code_warntype alphaFilter(Vi);

Variables
  #self#::Core.Compiler.Const(alphaFilter, false)
  V::Array{Float64,2}

Body::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
1 ─ %1 = Core.apply_type(Main.Array, Main.Int64, 1)::Core.Compiler.Const(Array{Int64,1}, false)
│   %2 = Base.getindex(%1)::Array{Array{Int64,1},1}
│   %3 = (#self#)(V, %2)::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
└──      return %3


In [45]:
filtration = alphaFilter(Vi);

In [46]:
@btime VV,EV,FV = alphaSimplex(Vi, filtration, 0.02) # 744.878 μs (41 allocations: 135.13 KiB)

  773.608 μs (41 allocations: 135.06 KiB)


3-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [991], [992], [993], [994], [995], [996], [997], [998], [999], [1000]]
 [[1, 401], [1, 687], [1, 731], [1, 893], [2, 187], [2, 435], [2, 496], [2, 849], [2, 974], [3, 121]  …  [943, 954], [943, 1000], [944, 972], [948, 990], [955, 967], [965, 998], [978, 991], [983, 985], [983, 991], [994, 1000]]
 [[1, 401, 893], [1, 687, 731], [1, 687, 893], [2, 187, 974], [2, 435, 496], [2, 496, 849], [2, 849, 974], [3, 121, 677], [3, 121, 707], [3, 145, 150]  …  [834, 855, 861], [834, 861, 902], [843, 849, 869], [854, 943, 954], [857, 888, 968], [903, 915, 977], [915, 940, 984], [928, 943, 954], [928, 943, 1000], [928, 994, 1000]]

In [47]:
@benchmark VV,EV,FV = alphaSimplex(Vi, filtration, 0.02)

BenchmarkTools.Trial: 
  memory estimate:  135.06 KiB
  allocs estimate:  41
  --------------
  minimum time:     745.439 μs (0.00% GC)
  median time:      877.626 μs (0.00% GC)
  mean time:        943.505 μs (1.85% GC)
  maximum time:     8.911 ms (88.86% GC)
  --------------
  samples:          5280
  evals/sample:     1

In [48]:
@code_warntype alphaSimplex(Vi, filtration, 0.02)

Variables
  #self#::Core.Compiler.Const(alphaSimplex, false)
  V::Array{Float64,2}
  filtration::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
  α_threshold::Float64
  #56::var"#56#57"
  dim::Int64
  simplexCollection::Array{Array{Array{Int64,1},1},1}
  @_8::Union{Nothing, Tuple{Pair{Array{Int64,1},Float64},DataStructures.SAIterationState}}
  k::Array{Int64,1}
  v::Float64
  @_11::Int64

Body::Array{_A,1} where _A
1 ─ %1  = Base.getproperty(Main.Lar, :Cells)::Any
│   %2  = Core.apply_type(Main.Array, %1, 1)::Type{Array{_A,1}} where _A
│         (dim = Main.size(V, 1))
│         (#56 = %new(Main.:(var"#56#57")))
│   %5  = #56::Core.Compiler.Const(var"#56#57"(), false)
│   %6  = (dim + 1)::Int64
│   %7  = (1:%6)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│   %8  = Base.Generator(%5, %7)::Core.Compiler.PartialStruct(Base.Generator{UnitRange{Int64},var"#56#57"}, Any[Core.Compiler.Const(var"#56#57"(), false), Core.Compiler.Parti

In [49]:
VV,EV,FV = alphaSimplex(Vi, filtration, 0.02)
points = [[p] for p in VV]
faces = [[f] for f in FV]
edges = [[e] for e in EV]
#GL.VIEW(
#    GL.GLExplode(Vi, [edges; faces], 1.5, 1.5, 1.5, 99, 1)
# );

filter_key = sort(unique(values(filtration)))

granular = 10

reduced_filter = filter_key[sort(abs.(rand(Int, granular).%length(filter_key)))]
reduced_filter = [reduced_filter; max(filter_key...)]

#
# Arlecchino's Lar
#
#
α = 0.0
for α in reduced_filter
    @show α
    @btime VV,EV,FV = alphaSimplex(Vi, filtration, α) # 315.970 μs (18 allocations: 21.00 KiB) in media
    #VV,EV,FV = alphaSimplex(Vi, filtration, α)

    #=
    GL.VIEW(
        GL.GLExplode(
            Vi,
            [[[f] for f in FV]; [[e] for e in EV]],
            1., 1., 1.,	# Explode Ratio
            99, 1		# Colors
        )
    )
    =#
    
end
#
#
# Appearing Colors
#

reduced_filter = [
    0.002;	0.003;	0.004;	0.005;  0.006;
    0.007;	0.008;	0.009;	0.010;	0.013;
    0.015;	0.020;	0.050;	1.000
]

i=2
for i = 2 : length(reduced_filter)
    @btime VV0, EV0, FV0 = alphaSimplex(Vi, filtration, reduced_filter[i-1])# 337.409 μs (36 allocations: 42.03 KiB) in media
    @btime VV,  EV,  FV  = alphaSimplex(Vi, filtration, reduced_filter[i])# 297.250 μs (32 allocations: 30.86 KiB) in media
    
    #=EV0mesh = GL.GLGrid(Vi, EV0)
    FV0mesh = GL.GLGrid(Vi, FV0)
    EVmesh = GL.GLGrid(Vi, setdiff(EV, EV0), GL.COLORS[2], 1)
    FVmesh = GL.GLGrid(Vi, setdiff(FV, FV0), GL.COLORS[7], 1)
    GL.VIEW([EV0mesh; FV0mesh; EVmesh; FVmesh])=#
end

α = 0.002737555276897019
  272.672 μs (18 allocations: 21.00 KiB)
α = 0.004414401743257116
  269.670 μs (18 allocations: 21.00 KiB)
α = 0.004509894422286172
  272.974 μs (18 allocations: 21.00 KiB)
α = 0.004864985819400807
  271.538 μs (18 allocations: 21.00 KiB)
α = 0.005000452214391951
  271.051 μs (18 allocations: 21.00 KiB)
α = 0.006180663883364004
  270.615 μs (18 allocations: 21.00 KiB)
α = 0.007175969075059448
  271.208 μs (18 allocations: 21.00 KiB)
α = 0.008060264934853813
  271.533 μs (18 allocations: 21.00 KiB)
α = 0.008118473695943824
  271.132 μs (18 allocations: 21.00 KiB)
α = 0.031204993754273103
  270.895 μs (18 allocations: 21.00 KiB)
α = 6.8875809878377
  270.566 μs (18 allocations: 21.00 KiB)
  296.968 μs (31 allocations: 26.80 KiB)
  325.195 μs (36 allocations: 41.84 KiB)
  294.467 μs (31 allocations: 26.80 KiB)
  323.675 μs (36 allocations: 41.84 KiB)
  291.317 μs (31 allocations: 26.80 KiB)
  322.464 μs (36 allocations: 41.84 KiB)
  293.263 μs (31 allocations: 26.

# Esempio 2D (utilizzando le funzioni del nuovo modulo)

In [50]:
"""
    pointsRand(V, VV, n, m)

Generate random points inside and otuside `(V, VV)`.
"""
function pointsRand(
        V::Lar.Points, EV::Lar.Cells, n = 1000, m = 0
    )::Tuple{Lar.Points, Lar.Points, Lar.Cells, Lar.Cells}
    classify = Lar.pointInPolygonClassification(V, EV)
    Vi = [0;0]
    Ve = [0;0]
    k1 = 0
    k2 = 0
    while k1 < n || k2 < m
        queryPoint = [rand();rand()]
        inOut = classify(queryPoint)

        if k1 < n && inOut == "p_in"
            Vi = hcat(Vi, queryPoint)
            k1 = k1 + 1;
        end
        if k2 < m && inOut == "p_out"
            Ve = hcat(Ve, queryPoint)
            k2 = k2 + 1;
        end
    end
    VVi = [[i] for i = 1 : n]
    VVe = [[i] for i = 1 : m]
    return Vi[:,2:end], Ve[:,2:end], VVi, VVe
end

filename = "../examples/examples2D/svg_files/Lar2.svg";

V,EV = Lar.svg2lar(filename);

Vi, Ve, VVi, VVe = pointsRand(V, EV, 1000, 10000);

#=
GL.VIEW([
 	GL.GLGrid(Vi, VVi, GL.COLORS[1], 1)
 	GL.GLGrid(Ve, VVe, GL.COLORS[12], 1)
])
=#

In [51]:
@btime filtration = AlphaShapes.alphaFilter(Vi); # 30.037 ms (283750 allocations: 24.19 MiB)

  29.218 ms (282031 allocations: 24.15 MiB)


In [52]:
@benchmark filtration = AlphaShapes.alphaFilter(Vi);

In [53]:
@code_warntype AlphaShapes.alphaFilter(Vi); 

Variables
  #self#::Core.Compiler.Const(Main.AlphaShapes.alphaFilter, false)
  V::Array{Float64,2}

Body::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
1 ─ %1 = Core.apply_type(Main.AlphaShapes.Array, Main.AlphaShapes.Int64, 1)::Core.Compiler.Const(Array{Int64,1}, false)
│   %2 = Base.getindex(%1)::Array{Array{Int64,1},1}
│   %3 = (#self#)(V, %2)::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
└──      return %3


In [54]:
filtration = AlphaShapes.alphaFilter(Vi); 

In [55]:
@btime VV,EV,FV = AlphaShapes.alphaSimplex(Vi, filtration, 0.02) # 690.470 μs (41 allocations: 135.00 KiB)

  641.624 μs (41 allocations: 135.00 KiB)


3-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [991], [992], [993], [994], [995], [996], [997], [998], [999], [1000]]
 [[1, 36], [1, 44], [1, 90], [1, 169], [1, 341], [1, 679], [1, 782], [2, 37], [2, 193], [2, 348]  …  [942, 994], [945, 956], [949, 952], [952, 985], [955, 989], [958, 999], [969, 983], [973, 982], [976, 991], [978, 986]]
 [[1, 36, 44], [1, 36, 169], [1, 44, 341], [1, 90, 679], [1, 90, 782], [1, 341, 782], [2, 37, 193], [2, 37, 729], [2, 193, 853], [2, 348, 572]  …  [807, 827, 902], [822, 863, 979], [834, 851, 932], [834, 851, 978], [834, 978, 986], [852, 858, 906], [854, 876, 937], [857, 874, 884], [866, 969, 983], [912, 973, 982]]

In [56]:
@benchmark VV,EV,FV = AlphaShapes.alphaSimplex(Vi, filtration, 0.02)

BenchmarkTools.Trial: 
  memory estimate:  135.00 KiB
  allocs estimate:  41
  --------------
  minimum time:     634.897 μs (0.00% GC)
  median time:      893.827 μs (0.00% GC)
  mean time:        1.075 ms (1.81% GC)
  maximum time:     10.740 ms (84.61% GC)
  --------------
  samples:          4644
  evals/sample:     1

In [57]:
@code_warntype AlphaShapes.alphaSimplex(Vi, filtration, 0.02)

Variables
  #self#::Core.Compiler.Const(Main.AlphaShapes.alphaSimplex, false)
  V::Array{Float64,2}
  filtration::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
  α_threshold::Float64
  #12::Main.AlphaShapes.var"#12#14"{Array{Float64,2},SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering},Float64}

Body::Array{Array{Array{Int64,1},1},1}
1 ─ %1  = LinearAlgebraicRepresentation.Cells::Core.Compiler.Const(Array{Array{Int64,1},1}, false)
│   %2  = Core.apply_type(Main.AlphaShapes.Array, %1, 1)::Core.Compiler.Const(Array{Array{Array{Int64,1},1},1}, false)
│   %3  = Main.AlphaShapes.:(var"#12#14")::Core.Compiler.Const(Main.AlphaShapes.var"#12#14", false)
│   %4  = Core.typeof(V)::Core.Compiler.Const(Array{Float64,2}, false)
│   %5  = Core.typeof(filtration)::Core.Compiler.Const(SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}, false)
│   %6  = Core.typeof(α_threshold)::Core.Compiler.Const(Float64, false)
│   %7  = Core.apply_type(%3, %4, %5, %6)::Core.Com

In [58]:
VV,EV,FV = AlphaShapes.alphaSimplex(Vi, filtration, 0.02)
points = [[p] for p in VV]
faces = [[f] for f in FV]
edges = [[e] for e in EV]
#GL.VIEW(
#    GL.GLExplode(Vi, [edges; faces], 1.5, 1.5, 1.5, 99, 1)
# );

filter_key = sort(unique(values(filtration)))

granular = 10

reduced_filter = filter_key[sort(abs.(rand(Int, granular).%length(filter_key)))]
reduced_filter = [reduced_filter; max(filter_key...)]

#
# Arlecchino's Lar
#
#
α = 0.0
for α in reduced_filter
    @show α
    @btime VV,EV,FV = AlphaShapes.alphaSimplex(Vi, filtration, α) # 255.389 μs (18 allocations: 21.00 KiB) in media
    #VV,EV,FV = alphaSimplex(Vi, filtration, α)

    #=
    GL.VIEW(
        GL.GLExplode(
            Vi,
            [[[f] for f in FV]; [[e] for e in EV]],
            1., 1., 1.,	# Explode Ratio
            99, 1		# Colors
        )
    )
    =#
    
end
#
#
# Appearing Colors
#

reduced_filter = [
    0.002;	0.003;	0.004;	0.005;  0.006;
    0.007;	0.008;	0.009;	0.010;	0.013;
    0.015;	0.020;	0.050;	1.000
]

i=2
for i = 2 : length(reduced_filter)
    @btime VV0, EV0, FV0 = AlphaShapes.alphaSimplex(Vi, filtration, reduced_filter[i-1]) # 292.120 μs (35 allocations: 40.78 KiB) in media
    @btime VV,  EV,  FV  = AlphaShapes.alphaSimplex(Vi, filtration, reduced_filter[i]) # 266.537 μs (30 allocations: 26.45 KiB) in media
    
    #=EV0mesh = GL.GLGrid(Vi, EV0)
    FV0mesh = GL.GLGrid(Vi, FV0)
    EVmesh = GL.GLGrid(Vi, setdiff(EV, EV0), GL.COLORS[2], 1)
    FVmesh = GL.GLGrid(Vi, setdiff(FV, FV0), GL.COLORS[7], 1)
    GL.VIEW([EV0mesh; FV0mesh; EVmesh; FVmesh])=#
end

α = 0.0014110027011969186
  238.783 μs (18 allocations: 21.00 KiB)
α = 0.0026418004289061516
  238.318 μs (18 allocations: 21.00 KiB)
α = 0.003162922206083835
  237.241 μs (18 allocations: 21.00 KiB)
α = 0.005226078953523071
  239.308 μs (18 allocations: 21.00 KiB)
α = 0.005460632096992462
  237.813 μs (18 allocations: 21.00 KiB)
α = 0.006572107856901798
  239.049 μs (18 allocations: 21.00 KiB)
α = 0.0074419284867178435
  241.433 μs (18 allocations: 21.00 KiB)
α = 0.008381134399334297
  240.929 μs (18 allocations: 21.00 KiB)
α = 0.00938028575171771
  241.554 μs (18 allocations: 21.00 KiB)
α = 0.012838178988897476
  240.820 μs (18 allocations: 21.00 KiB)
α = 16.224814728689086
  240.947 μs (18 allocations: 21.00 KiB)
  262.935 μs (33 allocations: 31.05 KiB)
  293.353 μs (36 allocations: 42.03 KiB)
  264.628 μs (33 allocations: 31.05 KiB)
  293.567 μs (36 allocations: 42.03 KiB)
  265.711 μs (33 allocations: 31.05 KiB)
  289.366 μs (36 allocations: 42.03 KiB)
  260.631 μs (33 allocations